In [1]:
import requests
import json
import pandas as pd
import numpy as np

import fiona
import descartes
from shapely.geometry import Point, Polygon
import statsmodels.formula.api as smf
import matplotlib.pylab as plt
import math
import copy
%matplotlib inline

/Users/yanxu/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import nltk
import string
import nltk.stem
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk.stem
from PIL import Image
import sys
import re
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/yanxu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## EDA on user reviews datasets

In [3]:
review=pd.read_csv("review_text.csv")
review=review.drop('Unnamed: 0', axis=1)
review.head()

,Review_ID,text,rating,time,Business_ID,type,ratings,geo_id,geometry
0,3berq4jHTbnJExYCloPJtA,"Tim is awesome, educated, charismatic, and fun...",5,2017-06-09 10:57:21,mMxsS8Y6bv0zCRJpFAhbEA,Education,5.0,41051005000,POINT (-122.687461853027 45.5287284851074)
1,i8KATzD28BBplS_vXPp94A,I am a brand new actor with my only on-camera ...,5,2014-05-14 19:25:28,mMxsS8Y6bv0zCRJpFAhbEA,Education,5.0,41051005000,POINT (-122.687461853027 45.5287284851074)
2,r00ER6z9bS3Mmhabp1Om6g,My daughter decided that she wanted to get int...,5,2017-11-13 09:28:56,19mIUTnqfUQUu6TguP_6Ng,Education,5.0,41051005000,POINT (-122.6879303 45.5266776)
3,FnH40b4lFXxvHiAaLXur8Q,Both my daughters have taken weekly classes an...,5,2017-01-31 19:26:20,19mIUTnqfUQUu6TguP_6Ng,Education,5.0,41051005000,POINT (-122.6879303 45.5266776)
4,F1o2s5zP2BtwvEuSr2mSzg,Barbara's acting class and one on one training...,5,2017-11-01 16:01:04,j0TR9ZTc4HhwhaNDBwWlrA,Education,5.0,41051005000,POINT (-122.688015 45.526672)


In [4]:
#Groupby the review text by GeoId
R=review[['geo_id','ratings','text','geometry']]
R.groupby('geo_id')['text'].apply(lambda x:x.str.cat(sep=':')).reset_index()

,geo_id,text
0,41051000100,Alan is the best!. He is so good with color. A...
1,41051000200,"Good price, friendly staff and a nice driving ..."
2,41051000302,Many of Portland's Public Schools are phenomen...
3,41051000401,I really like this place - it's always super c...
4,41051000502,The Green Tulip Preschool is a licensed in hom...
...,...,...
120,41051010304,I installed a new shower pan and needed a p-tr...
121,41051010408,It has been over 4 years for my dogs at DogTow...
122,41051010410,Wow. Precision Energy had an appointment sched...
123,41051010600,I wanted to take the time to praise the Oregon...


In [7]:
#get the review text by GeoId
dic = {}
total_text = []
for i in R.index:
    if R["geo_id"][i] in dic :
        total_text[dic[R["geo_id"][i]]] += R["text"][i]
    else:
        dic[R["geo_id"][i]] = len(total_text)
        total_text.append(R["text"][i])

total_text[0]

'Tim is awesome, educated, charismatic, and fun. Definitely a worth while experience for all acting fans at all levels!I am a brand new actor with my only on-camera and on-set experience being background, and Tim\'s Beginning On-Camera Workshop was the first acting class I\'d...My daughter decided that she wanted to get into television and radio commercials. Even though I work for a Television station it was difficult to get her...Both my daughters have taken weekly classes and many workshops with the coaches at Acting for Kids & Teens. This is a serious acting studio. The kids and...Barbara\'s acting class and one on one training are truly special and rare finds.  She has an uncanny ability to see what is going on in students and to...This music school provide their students with a well-rounded musical education. They believe  that the role of education should be to provide life-long...New Avenues INK is awesome! I used them to order t-shirts for my workplace. We really wanted to supp

In [8]:
#Clean text
del_words=['place','time','got','get','would','go','even','portland','one','first','day','area',
           'year','went','look','service','love','like','help','great','alway','servic',
           'always','years','best','came','really','new','little','location','hours','good',
           'amazing','experience','friendly']

def preprocess_data(data):
    # This function should return a list of lists of preprocessed tokens for each message
    preprocessed_text=[]
    for i in data:
        words = nltk.word_tokenize(i)
        words = [word.lower() for word in words if word.isalpha()] #clean punctuation 
        words = [word.lower() for word in words if re.match('^[a-zA-Z]+', word)]  
        english_stops = set(stopwords.words('english'))
        words = [word for word in words if word not in english_stops] 
        filtered_words = [word for word in words if word not in del_words]# clean stopwords
        preprocessed_text.append(filtered_words)
        
    return preprocessed_text

In [9]:
# Cout on word frequency
def count_words(s, n):
    """Word Frequency"""
 
    s_list = s 
    top_n_dict = {}
    for word in s_list:
        if word in top_n_dict:
            top_n_dict[word] += 1
        else:
            top_n_dict[word] = 1
 
    word_frequency = []
    values = sorted(list(set(top_n_dict.values())), reverse=True) 
    for w in values:
        word_list = []
        for k, v in top_n_dict.items():
            if v == w:
                word_list.append(k)

        word_frequency.extend(sorted(word_list))
 
    return word_frequency[:n]

In [10]:
review_sen=preprocess_data(total_text)

In [11]:
tokens=[]
for i in review_sen:
    token=count_words(i, 10)
    tokens.append(token)

In [13]:
Top10words= pd.DataFrame(tokens,dic)
Top10words

,0,1,2,3,4,5,6,7,8,9
41051005000,happy,food,many,staff,ever,car,never,super,take,awesome
41051005100,food,bar,hair,art,nice,many,store,found,pearl,around
41051001102,food,staff,show,happy,night,back,cocktails,could,everything,people
41051000901,ca,food,find,class,long,made,cute,helpful,shop,nice
41051010600,food,store,nice,downtown,hotel,ordered,never,found,us,bar
...,...,...,...,...,...,...,...,...,...,...
41051010303,beer,corner,selection,another,cold,long,market,markets,near,two
41051010304,needed,arrived,called,guys,leo,perfect,worked,added,advance,apologies
41051009903,manny,beyond,corners,cut,honest,put,quality,roof,satisfied,team
41051010410,marc,window,act,appointment,arrival,cares,class,clearly,clients,complicated


### Get word fruency on different types by GeoID

In [14]:
### by types
R_type=review[['geo_id','ratings','text','type']]
R_type.head()

,geo_id,ratings,text,type
0,41051005000,5.0,"Tim is awesome, educated, charismatic, and fun...",Education
1,41051005000,5.0,I am a brand new actor with my only on-camera ...,Education
2,41051005000,5.0,My daughter decided that she wanted to get int...,Education
3,41051005000,5.0,Both my daughters have taken weekly classes an...,Education
4,41051005000,5.0,Barbara's acting class and one on one training...,Education


In [15]:
types=R_type.groupby(['geo_id','type'])['text'].apply(lambda x:x.str.cat(sep=':')).reset_index()
types.head()

,geo_id,type,text
0,41051000100,Arts_Entertainment,Love this Museum\nOwners are the best \n\nBe s...
1,41051000100,Beauty&Spa,My hubby and I have been getting massages from...
2,41051000100,Car,This is a review for Daniel @ PDX car inspecti...
3,41051000100,Education,Alan is the best!. He is so good with color. A...
4,41051000100,Financial,"I was in need of a tax specialist, as I had tr..."


In [16]:
del_words=['place','time','got','get','would','go','even','portland','one','first','day','area',
           'year','went','look','service','help','alway','servic','could','almost','several','many'
           'always','years','came','really','little','location','hours','experience']

def preprocess_data(data):
    # This function should return a list of lists of preprocessed tokens for each message
    preprocessed_text=[]
    for i in data:
        words = nltk.word_tokenize(i)
        words = [word.lower() for word in words if word.isalpha()] #clean punctuation 
        words = [word.lower() for word in words if re.match('^[a-zA-Z]+', word)]  
        english_stops = set(stopwords.words('english'))
        filtered_words = [word for word in words if word not in english_stops] 
        filtered_words = [word for word in filtered_words if word not in del_words]# clean stopwords
        preprocessed_text.append(filtered_words)
        
    return preprocessed_text

In [17]:
text_type=types['text']
review_text=preprocess_data(text_type)

In [18]:
tokens_type=[]
for i in review_text:
    token=count_words(i, 5)
    tokens_type.append(token)

In [19]:
for i in tokens_type:
    words =tokens_type
types['top_words']= tokens_type
types=types[['geo_id','type','top_words']]
types

,geo_id,type,top_words
0,41051000100,Arts_Entertainment,"[fun, shop, art, every, oaks]"
1,41051000100,Beauty&Spa,"[hair, great, love, salon, best]"
2,41051000100,Car,"[car, brakes, bringing, daniel, dave]"
3,41051000100,Education,"[school, daycare, great, another, centers]"
4,41051000100,Financial,"[account, new, car, home, kevin]"
...,...,...,...
1043,41051980000,Arts_Entertainment,"[machines, space, amusement, bar, business]"
1044,41051980000,Car,"[king, yet, areas, axal, back]"
1045,41051980000,Home_Services,"[tice, electrical, electric, also, always]"
1046,41051980000,Shopping,"[blamed, buyer, customer, happy, left]"


In [20]:
pivot_table = types.pivot_table(index=['geo_id'],
                 columns=['type'],
                 values=['top_words'],
                 aggfunc=lambda x: ' '.join(str(v) for v in x))

In [21]:
pivot_table[20:30]

top_words  \
type                                        Arts_Entertainment   
geo_id                                                           
41051001701        ['great', 'drinks', 'good', 'two', 'ashes']   
41051001702    ['night', 'last', 'arrived', 'artists', 'arts']   
41051001801     ['show', 'amazing', 'curt', 'adults', 'jimmy']   
41051001802     ['james', 'safe', 'session', 'around', 'back']   
41051002000           ['back', 'great', 'live', 'good', 'old']   
41051002100  ['great', 'amazing', 'theater', 'aerial', 'best']   
41051002203                                                NaN   
41051002303       ['event', 'aerial', 'great', 'love', 'show']   
41051002401      ['wine', 'fun', 'provided', 'tour', 'coming']   
41051002402  ['halloween', 'buffet', 'comes', 'crazy', 'dec...   

                                                                \
type                                                Beauty&Spa   
geo_id                                                           
41051001701      ['hair', 'done', 'amazing', 'every', 'salon']   
41051001702       ['hair', 'salon', 'ago', 'great', 'haircut']   
41051001801       ['new', 'great', 'job', 'took', 'beautiful']   
41051001802  ['massage', 'natasha', 'work', 'amazing', 'best']   
41051002000    ['hair', 'massage', 'salon', 'amazing', 'best']   
41051002100     ['amazing', 'hair', 'best', 'massage', 'ever']   
41051002203     ['salon', 'hair', 'amazing', 'found', 'great']   
41051002303         ['arno', 'rhea', 'hair', 'kirin', 'sarah']   
41051002401  ['tell', 'able', 'also', 'anymore', 'appointme...   
41051002402       ['hair', 'amazing', 'new', 'float', 'going']   

                                                                \
type                                                       Car   
geo_id                                                           
41051001701       ['car', 'honest', 'work', 'always', 'great']   
41051001702        ['car', 'best', 'buying', 'honest', 'niko']   
41051001801   ['car', 'expense', 'gary', 'headache', 'helped']   
41051001802  ['accountability', 'actions', 'actually', 'aut...   
41051002000           ['car', 'told', 'took', 'bought', 'fix']   
41051002100  ['car', 'work', 'amazing', 'dealership', 'great']   
41051002203           ['car', 'auto', 'best', 'caleb', 'care']   
41051002303          ['car', 'great', 'change', 'oil', 'work']   
41051002401                                                NaN   
41051002402  ['called', 'windows', 'air', 'anymore', 'broken']   

                                                                \
type                                                 Education   
geo_id                                                           
41051001701  ['attended', 'class', 'drinks', 'mixology', 'a...   
41051001702  ['excellent', 'night', 'school', 'choice', 'ch...   
41051001801   ['david', 'school', 'spanish', 'beck', 'advice']   
41051001802  ['preschool', 'son', 'love', 'absolutely', 'am...   
41051002000  ['space', 'working', 'great', 'preschool', 'vi...   
41051002100  ['great', 'amazing', 'children', 'class', 'cla...   
41051002203            ['two', 'make', 'pest', 'us', 'booked']   
41051002303     ['bike', 'daycare', 'need', 'nephew', 'niece']   
41051002401  ['sam', 'teacher', 'amazing', 'great', 'irving...   
41051002402  ['school', 'preschool', 'teachers', 'learning'...   

                                                                \
type                                                 Financial   
geo_id                                                           
41051001701  ['customer', 'business', 'taxes', 'account', '...   
41051001702  ['insurance', 'able', 'accident', 'agent', 'al...   
41051001801  ['insurance', 'cox', 'business', 'accident', '...   
41051001802   ['agent', 'best', 'definitely', 'ever', 'great']   
41051002000   ['cccu', 'great', 'home', 'family', 'insurance']   
41051002100  ['jay', 'mariette', 'professional', 'bgo', 'kn...   
41051002203  ['answerin